In [2]:
# importing libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

import numpy as np
from matplotlib import pyplot as plt

import datetime
from dateutil.relativedelta import relativedelta


# Stays in hospitality

### INE stays in hospitality - https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_indicadores&indOcorrCod=0009808&contexto=bd&selTab=tab2


In [4]:
# Read file

path='ine_stays_hospitality_feb2021.csv'
df_stays = pd.read_csv(path, sep = ';')

df_stays.head()

,year,month,Continente,Norte,Centro,AML,Alentejo,Algarve
0,2020,12,801880,178378,150819,241202,75784,155697
1,2020,11,771053,162534,133473,216267,75523,183256
2,2020,10,1984498,402701,319477,399556,167704,695060
3,2020,9,3215366,560584,464646,507650,260222,1422264
4,2020,8,4742059,837872,765162,638851,405765,2094409


In [5]:
pd.crosstab(df_stays['year'], df_stays['month'])

month,1,2,3,4,5,6,7,8,9,10,11,12
year,,,,,,,,,,,,
2017,1,1,1,1,1,1,1,1,1,1,1,1
2018,1,1,1,1,1,1,1,1,1,1,1,1
2019,1,1,1,1,1,1,1,1,1,1,1,1
2020,1,1,1,1,1,1,1,1,1,1,1,1


In [6]:
# Transform data

df_stays = df_stays.melt(id_vars=['year', 'month'])
df_stays.columns = ['year', 'month', 'nuts', 'stays']

df_stays[['year', 'month']] = df_stays[['year', 'month']].astype(str)
df_stays['month'] = df_stays['month'].str.pad(2, side='left', fillchar='0')

df_stays['year_month'] = df_stays['year'] + df_stays['month']
df_stays['year_month'] = df_stays['year_month'].astype(int)

df_stays['year_month']= pd.to_datetime(df_stays.year_month, format='%Y%m') + pd.DateOffset(months=1) - datetime.timedelta(days=1) 
df_stays['year_month']= pd.to_datetime(df_stays.year_month, format='%Y%m')

df_stays.head()

,year,month,nuts,stays,year_month
0,2020,12,Continente,801880,2020-12-31
1,2020,11,Continente,771053,2020-11-30
2,2020,10,Continente,1984498,2020-10-31
3,2020,09,Continente,3215366,2020-09-30
4,2020,08,Continente,4742059,2020-08-31


In [7]:
#Interpolate

df_interpol_stays = pd.DataFrame()

for nut in df_stays.nuts.unique():  
    print(nut)
    df_temp = df_stays[df_stays['nuts'] == nut]
    df_temp = df_temp.set_index(['year_month'])

    rng = pd.date_range(df_temp.index.min(), df_temp.index.max(), freq='D')
    df_week = df_temp.reindex(rng)
    
    df_week['interpolated_stays'] = df_week.stays.interpolate()
    
    df_week = df_week.reset_index()
    df_week['year'] = pd.DatetimeIndex(df_week['index']).year
    df_week['nuts'] = nut
    
    
    df_interpol_stays = pd.concat([df_interpol_stays, df_week], axis = 0)
    
    
df_interpol_stays.head()

Continente
Norte
Centro
AML
Alentejo
Algarve


,index,year,month,nuts,stays,interpolated_stays
0,2017-01-31,2017,01,Continente,"2,112,718.00","2,112,718.00"
1,2017-02-01,2017,NaN,Continente,nan,"2,126,157.14"
2,2017-02-02,2017,NaN,Continente,nan,"2,139,596.29"
3,2017-02-03,2017,NaN,Continente,nan,"2,153,035.43"
4,2017-02-04,2017,NaN,Continente,nan,"2,166,474.57"


In [8]:
df_interpol_stays[df_interpol_stays['nuts'] == 'AML'].sort_values(['nuts','year','month']).head(20)

,index,year,month,nuts,stays,interpolated_stays
0,2017-01-31,2017,01,AML,"878,774.00","878,774.00"
28,2017-02-28,2017,02,AML,"920,076.00","920,076.00"
59,2017-03-31,2017,03,AML,"1,221,321.00","1,221,321.00"
89,2017-04-30,2017,04,AML,"1,506,142.00","1,506,142.00"
120,2017-05-31,2017,05,AML,"1,533,018.00","1,533,018.00"
150,2017-06-30,2017,06,AML,"1,572,130.00","1,572,130.00"
181,2017-07-31,2017,07,AML,"1,724,545.00","1,724,545.00"
212,2017-08-31,2017,08,AML,"1,898,121.00","1,898,121.00"
242,2017-09-30,2017,09,AML,"1,656,960.00","1,656,960.00"
273,2017-10-31,2017,10,AML,"1,603,352.00","1,603,352.00"


# Revenue hospitality

In [9]:
path='ine_revenue_hospitality_feb21.csv'
df_rev = pd.read_csv(path, sep = ';', encoding='Latin')
df_rev.head()

,year,month,1: Continente,11: Norte,16: Centro,17: Área Metropolitana de Lisboa,18: Alentejo,15: Algarve
0,2020,12,43464,9617,7657,13133,4936,8122
1,2020,11,38373,8100,5931,11606,4216,8519
2,2020,10,107296,20688,14883,24008,9621,38097
3,2020,9,187142,30964,23282,29302,16395,87199
4,2020,8,308663,46303,39507,38800,29420,154633


In [10]:
pd.crosstab(df_rev['year'], df_rev['month'])

month,1,2,3,4,5,6,7,8,9,10,11,12
year,,,,,,,,,,,,
2017,1,1,1,1,1,1,1,1,1,1,1,1
2018,1,1,1,1,1,1,1,1,1,1,1,1
2019,1,1,1,1,1,1,1,1,1,1,1,1
2020,1,1,1,1,1,1,1,1,1,1,1,1


In [11]:
# Transform data

df_rev = df_rev.melt(id_vars=['year', 'month'])
df_rev.columns = ['year', 'month', 'nuts', 'rev']

df_rev[['year', 'month']] = df_rev[['year', 'month']].astype(str)
df_rev['month'] = df_rev['month'].str.pad(2, side='left', fillchar='0')

df_rev['year_month'] = df_rev['year'] + df_rev['month']
df_rev['year_month'] = df_rev['year_month'].astype(int)

df_rev['year_month']= pd.to_datetime(df_rev.year_month, format='%Y%m') + pd.DateOffset(months=1) - datetime.timedelta(days=1) 
df_rev['year_month']= pd.to_datetime(df_rev.year_month, format='%Y%m')


df_rev.head()

,year,month,nuts,rev,year_month
0,2020,12,1: Continente,43464,2020-12-31
1,2020,11,1: Continente,38373,2020-11-30
2,2020,10,1: Continente,107296,2020-10-31
3,2020,09,1: Continente,187142,2020-09-30
4,2020,08,1: Continente,308663,2020-08-31


In [12]:
df_rev['nuts'] = df_rev['nuts'].str.strip('1234567891911: ')
df_rev.loc[df_rev['nuts'] == 'Área Metropolitana de Lisboa', 'nuts'] = 'AML'
df_rev['nuts'].unique()

array(['Continente', 'Norte', 'Centro', 'AML', 'Alentejo', 'Algarve'],
      dtype=object)

In [13]:
#Interpolate

df_interpol_rev = pd.DataFrame()

for nut in df_rev.nuts.unique():   
    df_temp = df_rev[df_rev['nuts'] == nut]
    df_temp = df_temp.set_index(['year_month'])

    rng = pd.date_range(df_temp.index.min(), df_temp.index.max(), freq='D')

    df_week = df_temp.reindex(rng)
    
    df_week['interpolated_rev'] = df_week.rev.interpolate()
    
    df_week = df_week.reset_index()
    df_week['year'] = pd.DatetimeIndex(df_week['index']).year
    df_week['nuts'] = nut
    
    df_interpol_rev = pd.concat([df_interpol_rev, df_week], axis = 0)

df_interpol_rev.head()

,index,year,month,nuts,rev,interpolated_rev
0,2017-01-31,2017,01,Continente,"106,197.00","106,197.00"
1,2017-02-01,2017,NaN,Continente,nan,"106,684.32"
2,2017-02-02,2017,NaN,Continente,nan,"107,171.64"
3,2017-02-03,2017,NaN,Continente,nan,"107,658.96"
4,2017-02-04,2017,NaN,Continente,nan,"108,146.29"


In [14]:
df_interpol = pd.merge(df_interpol_stays[['index', 'year', 'nuts', 'stays', 'interpolated_stays']], df_interpol_rev[['index', 'year', 'nuts', 'rev', 'interpolated_rev']], how='left', left_on=['index', 'year', 'nuts'], right_on=['index', 'year', 'nuts'])
df_interpol.head(100)

,index,year,nuts,stays,interpolated_stays,rev,interpolated_rev
0,2017-01-31,2017,Continente,"2,112,718.00","2,112,718.00","106,197.00","106,197.00"
1,2017-02-01,2017,Continente,nan,"2,126,157.14",nan,"106,684.32"
2,2017-02-02,2017,Continente,nan,"2,139,596.29",nan,"107,171.64"
3,2017-02-03,2017,Continente,nan,"2,153,035.43",nan,"107,658.96"
4,2017-02-04,2017,Continente,nan,"2,166,474.57",nan,"108,146.29"
5,2017-02-05,2017,Continente,nan,"2,179,913.71",nan,"108,633.61"
6,2017-02-06,2017,Continente,nan,"2,193,352.86",nan,"109,120.93"
7,2017-02-07,2017,Continente,nan,"2,206,792.00",nan,"109,608.25"
8,2017-02-08,2017,Continente,nan,"2,220,231.14",nan,"110,095.57"
9,2017-02-09,2017,Continente,nan,"2,233,670.29",nan,"110,582.89"


In [16]:
# Write CSV

df_interpol.to_csv('ine_stays_revune_hospitality_interpol.csv')

# Business volume hospitality/restaurants

In [17]:
path='ine_hospitality_business_vol_feb21.csv'
df_vol = pd.read_csv(path, sep = ';', decimal=',')
df_vol.head()

,year,month,"Alojamento, restauracao e similares",Alojamento,Restauracao
0,2020,12,-44.60,-66.80,-36.50
1,2020,11,-41.00,-63.20,-33.00
2,2020,10,-37.20,-58.60,-29.50
3,2020,9,-33.50,-52.00,-26.80
4,2020,8,-29.50,-44.40,-24.10


In [18]:
pd.crosstab(df_vol['year'], df_vol['month'])

month,1,2,3,4,5,6,7,8,9,10,11,12
year,,,,,,,,,,,,
2010,1,1,1,1,1,1,1,1,1,1,1,1
2011,1,1,1,1,1,1,1,1,1,1,1,1
2012,1,1,1,1,1,1,1,1,1,1,1,1
2013,1,1,1,1,1,1,1,1,1,1,1,1
2014,1,1,1,1,1,1,1,1,1,1,1,1
2015,1,1,1,1,1,1,1,1,1,1,1,1
2016,1,1,1,1,1,1,1,1,1,1,1,1
2017,1,1,1,1,1,1,1,1,1,1,1,1
2018,1,1,1,1,1,1,1,1,1,1,1,1


In [19]:
# Transform data 

df_vol[['year', 'month']] = df_vol[['year', 'month']].astype(str)
df_vol['month'] = df_vol['month'].str.pad(2, side='left', fillchar='0')

df_vol['year_month'] = df_vol['year'] + df_vol['month']
df_vol['year_month'] = df_vol['year_month'].astype(int)

df_vol['year_month']= pd.to_datetime(df_vol.year_month, format='%Y%m') + pd.DateOffset(months=1) - datetime.timedelta(days=1) 
df_vol['year_month']= pd.to_datetime(df_vol.year_month, format='%Y%m')


df_vol.head()

,year,month,"Alojamento, restauracao e similares",Alojamento,Restauracao,year_month
0,2020,12,-44.60,-66.80,-36.50,2020-12-31
1,2020,11,-41.00,-63.20,-33.00,2020-11-30
2,2020,10,-37.20,-58.60,-29.50,2020-10-31
3,2020,09,-33.50,-52.00,-26.80,2020-09-30
4,2020,08,-29.50,-44.40,-24.10,2020-08-31


In [20]:
#Interpolate

df_temp = df_vol.copy()
df_temp = df_temp.set_index(['year_month'])

rng = pd.date_range(df_temp.index.min(), df_temp.index.max(), freq='D')

df_interpol_vol = df_temp.reindex(rng)
df_interpol_vol['interpolated_host'] = df_interpol_vol['Alojamento'].interpolate()
df_interpol_vol['interpolated_rest'] = df_interpol_vol['Restauracao'].interpolate()
df_interpol_vol['interpolated_host_rest'] = df_interpol_vol['Alojamento, restauracao e similares'].interpolate()

df_interpol_vol = df_interpol_vol.reset_index()
df_interpol_vol['year'] = pd.DatetimeIndex(df_interpol_vol['index']).year
    
df_interpol_vol.tail(32)

,index,year,month,"Alojamento, restauracao e similares",Alojamento,Restauracao,interpolated_host,interpolated_rest,interpolated_host_rest
3956,2020-11-30,2020,11,-41.00,-63.20,-33.00,-63.20,-33.00,-41.00
3957,2020-12-01,2020,NaN,nan,nan,nan,-63.32,-33.11,-41.12
3958,2020-12-02,2020,NaN,nan,nan,nan,-63.43,-33.23,-41.23
3959,2020-12-03,2020,NaN,nan,nan,nan,-63.55,-33.34,-41.35
3960,2020-12-04,2020,NaN,nan,nan,nan,-63.66,-33.45,-41.46
3961,2020-12-05,2020,NaN,nan,nan,nan,-63.78,-33.56,-41.58
3962,2020-12-06,2020,NaN,nan,nan,nan,-63.90,-33.68,-41.70
3963,2020-12-07,2020,NaN,nan,nan,nan,-64.01,-33.79,-41.81
3964,2020-12-08,2020,NaN,nan,nan,nan,-64.13,-33.90,-41.93
3965,2020-12-09,2020,NaN,nan,nan,nan,-64.25,-34.02,-42.05


In [22]:
# Write CSV

df_interpol_vol.to_csv('ine_hospitality_business_volume_interpol.csv')